In [107]:
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
import pandas as pd
from sklearn.utils import check_random_state
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold
from sklearn.utils import shuffle

In [128]:
n = 20
x = np.arange(n)
rs = check_random_state(0)
y = rs.randint(-10, 10, size=(n,)) + 50. * np.log(1 + np.arange(n))
x = x.reshape(-1, 1)
y = y.reshape(-1, 1)

In [129]:
pf = PolynomialFeatures()
lr = LinearRegression()
pl = Pipeline([('pf', pf), ('lr', lr)])
x, y = shuffle(x, y)
x_tr, x_test, y_tr, y_test = train_test_split(x, y, test_size=0.33)
n_folds = 5
degrees = range(0, 10)
kf = KFold(x_tr.shape[0], n_folds)
tr_scores = np.zeros((n_folds, len(degrees)))
cv_scores = np.zeros((n_folds, len(degrees)))
for i, (tr_ind, cv_ind) in enumerate(kf):
    for j, d in enumerate(degrees):
        pl.set_params(pf__degree = d).fit(x_tr[tr_ind], y_tr[tr_ind])
        tr_scores[i, j] = pl.score(x_tr[tr_ind], y_tr[tr_ind])
        cv_scores[i, j] = pl.score(x_tr[cv_ind], y_tr[cv_ind])
tr_scores = tr_scores.mean(axis=0)
cv_scores = cv_scores.mean(axis=0)

In [130]:
cv_scores

array([ -1.54396159e+00,   3.95091101e-01,   7.50568551e-01,
         6.76287009e-01,   6.89717985e-01,   7.35884690e-01,
         1.58794618e-01,   8.37294314e-01,  -1.24330794e+02,
        -1.04037600e+03])

In [131]:
d_ = np.argmax(cv_scores)
pl.set_params(pf__degree = d_)
pl.fit(x_tr, y_tr)
score_ = pl.score(x_test, y_test)

In [132]:
plt.plot(degrees, cv_scores, 'r-')
plt.plot(degrees, tr_scores, 'b-')
plt.scatter(d_, score_)
plt.grid(True)
plt.show()

In [121]:
plt.plot(x, y, 'r.')
plt.grid(True)
plt.show()
